## 问题定义

本问题为求解2D Helmholtz方程。其形式可以参考案例18中的PDE定义。


### 求解目标

要求求解给定区域内的Helmholtz方程

## First Order

对于高阶导数，可以拆解为两步低阶。在一些特殊方法以及PDE求解问题中，高阶导数项会带来不稳定，通过引入额外的约束项能够缓解这一问题。

具体而言，对于Helmholtz有如下拆解：

![First Order](./resource/first_order.png)


## 求解

In [1]:
import os
import warnings

# optional
# set appropriate GPU in case of multi-GPU machine
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [2]:
from modulus.sym.hydra import to_yaml
from modulus.sym.hydra.utils import compose
from modulus.sym.utils.io.plotter import ValidatorPlotter, InferencerPlotter

In [3]:
import torch
import numpy as np
from sympy import Symbol, pi, sin
from typing import List, Tuple, Dict

import modulus.sym
from modulus.sym.hydra import to_absolute_path, instantiate_arch, ModulusConfig
from modulus.sym.utils.io import csv_to_dict
from modulus.sym.solver import Solver
from modulus.sym.domain import Domain
from modulus.sym.geometry.primitives_2d import Rectangle
from modulus.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)
from modulus.sym.domain.validator import PointwiseValidator
from modulus.sym.key import Key
from modulus.sym.node import Node
from modulus.sym.geometry.adf import ADF
from modulus.sym.eq.pdes.wave_equation import HelmholtzEquation

In [4]:
cfg = compose(config_path="conf", config_name="config")
cfg.network_dir = 'outputs'    # Set the network directory for checkpoints
print(to_yaml(cfg))

/usr/local/lib/python3.10/dist-packages/modulus/sym/hydra/utils.py:148: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(


training:
  max_steps: 20000
  grad_agg_freq: 1
  rec_results_freq: 1000
  rec_validation_freq: ${training.rec_results_freq}
  rec_inference_freq: ${training.rec_results_freq}
  rec_monitor_freq: ${training.rec_results_freq}
  rec_constraint_freq: 2000
  save_network_freq: 1000
  print_stats_freq: 100
  summary_freq: 1000
  amp: false
  amp_dtype: float16
  ntk:
    use_ntk: false
    save_name: null
    run_freq: 1000
graph:
  func_arch: false
  func_arch_allow_partial_hessian: true
stop_criterion:
  metric: null
  min_delta: null
  patience: 50000
  mode: min
  freq: 1000
  strict: false
profiler:
  profile: false
  start_step: 0
  end_step: 100
  name: nvtx
network_dir: outputs
initialization_network_dir: ''
save_filetypes: vtk,npz
summary_histograms: false
jit: true
jit_use_nvfuser: true
jit_arch_mode: only_activation
jit_autograd_nodes: false
cuda_graphs: false
cuda_graph_warmup: 20
find_unused_parameters: false
broadcast_buffers: false
device: ''
debug: false
run_mode: train
arch

### 定义必要组件

#### Geo

In [5]:
x, y = Symbol("x"), Symbol("y")
height = 2
width = 2
rec = Rectangle((-width / 2, -height / 2), (width / 2, height / 2))

#### PDE

In [6]:
wave = HelmholtzEquation(u="u", k=1.0, dim=2, mixed_form=True)  # 亥姆霍兹方程，注意这里需要定义mixed_form=True，从而启用first order method

#### Model

In [7]:
# 定义简单的全连接网络
# 输入为空间坐标
# 输出为模型预测结果以及对应的一阶项（关于此，请参考官方文档中的first order method）
# https://docs.nvidia.com/deeplearning/modulus/modulus-sym/user_guide/theory/advanced_schemes.html#exact-boundary-condition-imposition
wave_net = instantiate_arch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("u"), Key("u_x"), Key("u_y")],
    cfg=cfg.arch.fully_connected,
)

#### Node

In [8]:
nodes = (
    wave.make_nodes()  # PDE
    + [wave_net.make_node(name="wave_network")]  # model
)

#### Domain

In [9]:
domain = Domain()

#### ABC边界

In [10]:
wall = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"u": 0},
    batch_size=cfg.batch_size.wall,
)
domain.add_constraint(wall, "wall")

#### PDE约束

In [11]:
interior = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={
        "helmholtz": -(
            -((pi) ** 2) * sin(pi * x) * sin(4 * pi * y)
            - ((4 * pi) ** 2) * sin(pi * x) * sin(4 * pi * y)
            + 1 * sin(pi * x) * sin(4 * pi * y)
        ),  # PDE约束
        "compatibility_u_x": 0,  # 一阶项预测约束
        "compatibility_u_y": 0,  # 一阶项预测约束
    },
    batch_size=cfg.batch_size.interior,
    bounds={x: (-width / 2, width / 2), y: (-height / 2, height / 2)},
    lambda_weighting={
        "helmholtz": Symbol("sdf"),
        "compatibility_u_x": 0.5,
        "compatibility_u_y": 0.5,
    },
)
domain.add_constraint(interior, "interior")

#### 验证器以及其他必要组件

In [12]:
file_path = "openfoam/helmholtz.csv"
if os.path.exists(to_absolute_path(file_path)):
    mapping = {"x": "x", "y": "y", "z": "u"}
    openfoam_var = csv_to_dict(to_absolute_path(file_path), mapping)
    openfoam_invar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["x", "y"]
    }
    openfoam_outvar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["u"]
    }

    openfoam_validator = PointwiseValidator(
        nodes=nodes,
        invar=openfoam_invar_numpy,
        true_outvar=openfoam_outvar_numpy,
        batch_size=1024,
        plotter=ValidatorPlotter(),
    )
    domain.add_validator(openfoam_validator)
else:
    warnings.warn(
        f"Directory {file_path} does not exist. Will skip adding validators. Please download the additional files from NGC https://catalog.ngc.nvidia.com/orgs/nvidia/teams/modulus/resources/modulus_sym_examples_supplemental_materials"
    )

### 求解器以及求解

In [13]:
# 定义求解器
slv = Solver(cfg, domain)

手动加载日志系统

In [14]:
import logging
# logging.getLogger().addHandler(logging.StreamHandler())
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

启动求解

In [15]:
slv.solve()

2024-03-03 08:32:50,927 - WARNING - Installed PyTorch version 2.2.0a0+81ea7a4 is not TorchScript supported in Modulus. Version 2.1.0a0+4136153 is officially supported.
2024-03-03 08:32:50,938 - INFO - attempting to restore from: /workspace/23_2D_Helmholtz_FirstOrder/outputs
2024-03-03 08:32:50,939 - WARNING - optimizer checkpoint not found
2024-03-03 08:32:50,940 - WARNING - model wave_network.0.pth not found
2024-03-03 08:32:51,965 - INFO - [step:          0] record constraint batch time:  5.973e-02s
2024-03-03 08:33:03,915 - INFO - [step:          0] record validators time:  1.195e+01s
2024-03-03 08:33:04,006 - INFO - [step:          0] saved checkpoint to /workspace/23_2D_Helmholtz_FirstOrder/outputs
2024-03-03 08:33:04,008 - INFO - [step:          0] loss:  1.050e+04
2024-03-03 08:33:08,897 - INFO - [step:        100] loss:  9.863e+03, time/iteration:  4.889e+01 ms
2024-03-03 08:33:13,665 - INFO - [step:        200] loss:  2.970e+03, time/iteration:  4.766e+01 ms
2024-03-03 08:33:1

2024-03-03 08:39:28,716 - INFO - [step:       6000] loss:  6.006e-02, time/iteration:  1.723e+02 ms
2024-03-03 08:39:34,111 - INFO - [step:       6100] loss:  2.005e-01, time/iteration:  5.395e+01 ms
2024-03-03 08:39:39,782 - INFO - [step:       6200] loss:  1.781e-01, time/iteration:  5.667e+01 ms
2024-03-03 08:39:45,645 - INFO - [step:       6300] loss:  1.430e-01, time/iteration:  5.862e+01 ms
2024-03-03 08:39:51,524 - INFO - [step:       6400] loss:  6.306e-02, time/iteration:  5.878e+01 ms
2024-03-03 08:39:57,343 - INFO - [step:       6500] loss:  3.494e-01, time/iteration:  5.819e+01 ms
2024-03-03 08:40:02,931 - INFO - [step:       6600] loss:  8.866e-01, time/iteration:  5.586e+01 ms
2024-03-03 08:40:08,724 - INFO - [step:       6700] loss:  6.045e-02, time/iteration:  5.790e+01 ms
2024-03-03 08:40:14,488 - INFO - [step:       6800] loss:  9.466e-02, time/iteration:  5.762e+01 ms
2024-03-03 08:40:20,032 - INFO - [step:       6900] loss:  2.369e-01, time/iteration:  5.543e+01 ms


2024-03-03 08:46:47,517 - INFO - [step:      12700] loss:  3.324e-02, time/iteration:  5.440e+01 ms
2024-03-03 08:46:53,253 - INFO - [step:      12800] loss:  6.092e-02, time/iteration:  5.734e+01 ms
2024-03-03 08:46:59,113 - INFO - [step:      12900] loss:  3.860e-02, time/iteration:  5.859e+01 ms
2024-03-03 08:47:16,853 - INFO - [step:      13000] record validators time:  1.126e+01s
2024-03-03 08:47:16,932 - INFO - [step:      13000] saved checkpoint to /workspace/23_2D_Helmholtz_FirstOrder/outputs
2024-03-03 08:47:16,934 - INFO - [step:      13000] loss:  3.245e-02, time/iteration:  1.782e+02 ms
2024-03-03 08:47:22,496 - INFO - [step:      13100] loss:  4.174e-02, time/iteration:  5.560e+01 ms
2024-03-03 08:47:28,256 - INFO - [step:      13200] loss:  4.364e-02, time/iteration:  5.759e+01 ms
2024-03-03 08:47:34,154 - INFO - [step:      13300] loss:  3.873e-02, time/iteration:  5.895e+01 ms
2024-03-03 08:47:39,956 - INFO - [step:      13400] loss:  3.496e-02, time/iteration:  5.801e+

2024-03-03 08:54:15,005 - INFO - [step:      19200] loss:  2.644e-02, time/iteration:  4.968e+01 ms
2024-03-03 08:54:20,773 - INFO - [step:      19300] loss:  2.577e-02, time/iteration:  5.767e+01 ms
2024-03-03 08:54:26,654 - INFO - [step:      19400] loss:  2.466e-02, time/iteration:  5.879e+01 ms
2024-03-03 08:54:32,303 - INFO - [step:      19500] loss:  2.617e-02, time/iteration:  5.647e+01 ms
2024-03-03 08:54:38,220 - INFO - [step:      19600] loss:  2.467e-02, time/iteration:  5.915e+01 ms
2024-03-03 08:54:44,103 - INFO - [step:      19700] loss:  2.595e-02, time/iteration:  5.881e+01 ms
2024-03-03 08:54:49,271 - INFO - [step:      19800] loss:  2.511e-02, time/iteration:  5.167e+01 ms
2024-03-03 08:54:55,100 - INFO - [step:      19900] loss:  2.421e-02, time/iteration:  5.827e+01 ms
2024-03-03 08:55:01,515 - INFO - [step:      20000] record constraint batch time:  6.141e-02s
2024-03-03 08:55:12,907 - INFO - [step:      20000] record validators time:  1.139e+01s
2024-03-03 08:55:1

### 后处理以及可视化

对于jupyter，比较方便的方法是使用matplotlib

此外，还可以使用tensorboard以及Paraview

![u](./outputs/validators/validator_u.png)